# 타이타닉 데이터셋
- https://www.kaggle.com/competitions/titanic/data
- https://www.kaggle.com/code/subinium/subinium-tutorial-titanic-beginner/notebook

## 1. 라이브러리 블러오기

In [37]:
# 분석 관련
import pandas as pd
import numpy as np

# 시각화 관련
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# Scikit-Learn의 다양한 머신러닝 모듈
# 분류 알고리즘 중에서 선형회귀, 서포트벡터머신, 랜덤포레스트, K-최근접 이웃 알고리즘
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

## 데이터 읽기

In [38]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# 데이터 미리보기
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


훈련 데이터와 테스트 데이터의 정보 확인해보기

In [39]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [40]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


위 결과에서 각각의 데이터 개수는 891개, 418개인 것을 확인할 수 있다. 특성은 각각 12개, 11개입니다. 그 이유는 훈련 데이터는 생존 여부를 알고 있기 때문입니다.

여기서 주의깊게 봐야할 부분은 다음과 같습니다.
- 각 데이터는 빈 부분이 있는가?
  - 빈 부분이 있다면, drop할 것인가 아니면 default값으로 채워넣을 것인가
  - cabin, Age, Embarked 세 항목에 주의
- 데이터는 float64로 변환할 수 있는가
  - 아니면 범주형 데이터로 만들 수 있는가

필요없는 부분이라고 생각되는 부분을 지웁니다. 여기서 PassengerID와 이름, 티켓을 지웁니다. 이름과 티켓에서 가져올 수 있는 데이터는 없기 때문입니다. 하지만 이 문제에서 결과물은 `'PassengerId', 'Survived'` 요소가 필요하므로 훈련데이터에서만 삭제합니다.

In [41]:
train_df = train_df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
test_df = test_df.drop(['Name', 'Ticket'], axis=1)

## 3.데이터 하나하나 처리하기
이제 남은 데이터 종류는 다음과 같습니다.

1. Pclass
2. Sex
3. Age
4. SibSp
5. Parch
6. Fare
7. Cabin
8. Embarked

이제 순서대로 보도록 하겠습니다.

### 3.1 Pclass
Pclass는 서수형 데이터입니다. 1등석, 2등석, 3등석과 같은 정보입니다. 처음에 확인시에 데이터가 비어있지 않은 것을 확인할 수 있었습니다.

데이터에 대한 확인과 데이터를 변환해보도록 하겠습니다. 우선 각 unique한 value에 대한 카운팅은 `value_counts()`메서드로 확인할 수 있습니다.

In [42]:
train_df['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

1, 2, 3은 정수이니 그냥 실수로만 바꾸면 되지않을까 생각할 수 있습니다. 하지만 1, 2, 3등급은 경우에 따라 다를 수 있지만 연속적인 정보가 아니며, 각 차이 또한 균등하지 않습니다. 그렇기에 범주형(카테고리) 데이터로 인식하고 인코딩해야합니다. (비슷한 예시로 영화 별점 등이 있습니다.)

이 데이터는 범주형 데이터이므로 one-hot-encoding을 `pd.get_dummies()` 메서드로 인코딩합시다.

In [43]:
pclass_train_dummies = pd.get_dummies(train_df['Pclass'])
pclass_test_dummies = pd.get_dummies(test_df['Pclass'])

train_df.drop(['Pclass'], axis=1, inplace=True)
test_df.drop(['Pclass'], axis=1, inplace=True)

train_df = train_df.join(pclass_train_dummies)
test_df = test_df.join(pclass_test_dummies)

이렇게 Pclass의 원본을 없애고, 범주형으로 개별로 데이터가 변환되었습니다.
> 여기서 살짝 실수한게 columns의 이름을 설정하고, 넣어줘야하는데 그렇지 않아 1, 2, 3이라는 컬럼으로 데이터가 들어갔습니다. 다른 데이터에는 이런 적용을 피하도록 합시다.

### 3.2 Sex
sex는 성별입니다. 남과 여로 나뉘므로 이 또한 one-hot-encoding을 진행해봅시다.

In [44]:
sex_train_dummies = pd.get_dummies(train_df['Sex'])
sex_test_dummies = pd.get_dummies(test_df['Sex'])

sex_train_dummies.columns = ['Female', 'Male']
sex_test_dummies.columns = ['Female', 'Male']

train_df.drop(['Sex'], axis=1, inplace=True)
test_df.drop(['Sex'], axis=1, inplace=True)

train_df = train_df.join(sex_train_dummies)
test_df = test_df.join(sex_test_dummies)

### 3.3 Age
나이는 연속형 데이터이므로, 큰 처리가 필요없습니다. (카테고리화를 하여 일부 알고리즘에서는 더 유용한 결과를 만들 수 있습니다.) 하지만 일부 NaN 데이터가 있으니 이를 채울 수 이쓴ㄴ 방법에 대해서 생각해봅시다.

1. 랜덤
2. 평균값
3. 중간값
4. 데이터 버리기

일단은 평균값으로 채우도록 하겠습니다. 데이터의 통일성을 가지기 위해 train 데이터셋의 평균값으로 훈련, 테스트 데이터셋을 채우겠습니다.

In [45]:
train_df['Age'].fillna(train_df['Age'].mean(), inplace=True)
test_df['Age'].fillna(test_df['Age'].mean(), inplace=True)

### 3.4 SibSp & Panch
형제 자메와 부모님은 가족으로 함께 처리할 수 있습니다. 하지만 마찬가지로 바꿀 필요는 없습니다.

### 3.5 Fare
Fare은 탑승료입니다. 신기하게 test 데이터셋에 1개의 데이터가 비어있습니다. 아마 디카프리오인듯 합니다. 우선 빈 부분을 `fillna` 메서드로 채우겠습니다.

저는 데이터 누락이 아닌 무단 탑승이라 생각하고 0으로 입력하겠습니다.

In [46]:
test_df['Fare'].fillna(0, inplace=True)

### 3.6 Cabin
Cabin은 객실입니다. NaN이 대부분인 데이터이므로 버립시다. 이 데이터를 살리는 것은 너무 어려운 일입니다.

In [47]:
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

### 3.7 Embarked
Embarked는 탑승 항구를 의미합니다. 우선 데이터를 확인해보겠습니다.

In [48]:
train_df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [49]:
test_df['Embarked'].value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

S가 대다수이고 일부 데이터가 비어있는 것을 알 수 있습니다. 빈 부분을 S로 우선 채우고 시작합시다.

In [50]:
train_df['Embarked'].fillna('S', inplace=True)
test_df['Embarked'].fillna('S', inplace=True)

In [51]:
embarked_train_dummies = pd.get_dummies(train_df['Embarked'])
embarked_test_dummies = pd.get_dummies(test_df['Embarked'])

embarked_train_dummies.columns = ['S', 'C', 'Q']
embarked_test_dummies.columns = ['S', 'C', 'Q']

train_df.drop(['Embarked'], axis=1, inplace=True)
test_df.drop(['Embarked'], axis=1, inplace=True)

train_df = train_df.join(embarked_train_dummies)
test_df = test_df.join(embarked_test_dummies)

### 4. 데이터 나누기
이제 학습용 데이터를 위해 데이터를 나누어야합니다.
`(정보, 생존 여부)`와 같은 형태를 위하여 다음과 같이 데이터를 나눕니다.

In [52]:
X_train = train_df.drop('Survived', axis=1)
Y_train = train_df['Survived']
X_test = test_df.drop('PassengerId', axis=1).copy()

### 5. 머신러닝 알고리즘 적용하기
이제 로지스틱 회귀, SVC, 랜덤 포레스트, K-최근접 이웃 알고리즘을 각각 적용해봅시다.

In [53]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_test)
logreg.score(X_train, Y_train)

/Users/hoseong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hoseong/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/hoseong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An erro

0.8058361391694725

In [54]:
# Support Vector Machines

svc = SVC()

svc.fit(X_train, Y_train)

Y_pred = svc.predict(X_test)

svc.score(X_train, Y_train)

/Users/hoseong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hoseong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hoseong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.6868686868686869

In [55]:
# Random Forests

random_forest = RandomForestClassifier(n_estimators=100)

random_forest.fit(X_train, Y_train)

Y_pred = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)

/Users/hoseong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hoseong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hoseong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.9820426487093153

In [56]:
knn = KNeighborsClassifier(n_neighbors = 3)

knn.fit(X_train, Y_train)

Y_pred = knn.predict(X_test)

knn.score(X_train, Y_train)

/Users/hoseong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hoseong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hoseong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.8361391694725028

### 6. 제출용 파일 만들기
랜덤 포레스트가 가장 좋은 결과를 내는 것을 알 수 있습니다. 그 결과로 submission 파일을 만들어 제출해봅시다.

In [59]:
# Random Forests

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)

submission = pd.DataFrame({
    'PassengerId' : test_df['PassengerId'],
    'Survived' : Y_pred
})
submission.to_csv('titanic.csv', index=False)

/Users/hoseong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hoseong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hoseong/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
